In [ ]:
!pip install jsonlines

In [ ]:
import os
import pickle
import jsonlines
import pandas as pd
import numpy as np
import json
import copy
from tqdm import tqdm
from collections import defaultdict
import requests
import json, pickle


In [ ]:

id_map = json.load(open("handled/id_map.json"))
item_dict = json.load(open("handled/item2attributes.json", "r"))

print("len(id_map) =", len(id_map))
print("len(item_emb) =", len(item_dict))

print("샘플 id_map =", list(id_map.items())[:10])
print("샘플 item_emb keys =", list(item_dict.keys())[:10])


/content/drive/MyDrive/LLM_ESR_2023_v3/data/Books
Buffered data was truncated after reaching the output size limit.

In [ ]:

def load_dataset():
        '''Load train, validation, test dataset'''

        usernum = 0
        itemnum = 0
        User = defaultdict(list)    # default value is a blank list
        user_train = {}
        user_valid = {}
        user_test = {}

        f = open('handled/inter.txt', 'r')
        for line in f:
            u, i = line.rstrip().split(' ')
            u = int(u)
            i = int(i)
            usernum = max(u, usernum)
            itemnum = max(i, itemnum)
            User[u].append(i)

        for user in tqdm(User):
            nfeedback = len(User[user])
            if nfeedback < 3:
                user_train[user] = User[user]
                user_valid[user] = []
                user_test[user] = []
            else:
                user_train[user] = User[user][:-2]
                user_valid[user] = []
                user_valid[user].append(User[user][-2])
                user_test[user] = []
                user_test[user].append(User[user][-1])

        return user_train


In [ ]:
inter = load_dataset()

100%|██████████| 776369/776369 [00:02<00:00, 266475.95it/s]


In [ ]:
print(len(inter))

776369


In [ ]:
prompt_template = "The user has visited following fashions(item): \n<HISTORY> \nplease conclude the user's perference."

In [ ]:
id_map.keys()

dict_keys(['user2id', 'item2id', 'id2user', 'id2item', 'attribute2id', 'id2attribute', 'attributeid2num'])

In [ ]:
import copy

id2attr = id_map["id2attribute"]  # attr_id(int/str) -> attr_name

def get_item_text_from_attributes(iid: int) -> str:

    attr_ids = item_dict.get(str(iid)) or item_dict.get(iid) or []
    names = []
    for aid in attr_ids:
        name = id2attr.get(aid) or id2attr.get(str(aid))
        if name:
            names.append(name)
    return "; ".join(names) if names else f"[iid={iid}]"

def get_user_prompt(user, history):
    prompt_template = "The user has visited following fashions: \n<HISTORY> \nplease conclude the user's perference."

    parts = []


    for idx, iid in enumerate(history):
        item_str = get_item_text_from_attributes(iid)
        parts.append(item_str)

    hist_str = ", ".join(parts)
    if len(hist_str) > 8000:
        hist_str = hist_str[-8000:]

    return prompt_template.replace("<HISTORY>", hist_str)


In [ ]:
user_data = {}

for user, history in tqdm(inter.items()):
    user_data[user] = get_user_prompt(user, history)


100%|██████████| 776369/776369 [00:13<00:00, 57380.94it/s]


In [ ]:
print(user_data[37])

The user has visited following fashions: 
Books; Teen & Young Adult; Literature & Fiction, Books; Teen & Young Adult; Literature & Fiction, Books; Teen & Young Adult; Science Fiction & Fantasy, Books; Crafts, Hobbies & Home; Home Improvement & Design, Books; Crafts, Hobbies & Home; Home Improvement & Design, Books; Science & Math; Mathematics, Books; Education & Teaching; Schools & Teaching 
please conclude the user's perference.


In [ ]:
print(len(inter))

776369


In [ ]:
with open(os.path.join("handled/user_str.json"), "w") as f:
    json.dump(user_data, f)

In [ ]:
user_data = json.load(open("handled/user_str.json", "r"))

In [1]:
# 이 셀은 직접 실행하세요 (sk-로 시작하는 키를 넣기)
# %env OPENAI_API_KEY=your api key


In [ ]:
import os, requests, json

url = "https://api.openai.com/v1/embeddings"

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")  # 환경변수에서 불러오기

payload = {
   "model": "text-embedding-ada-002",   # 최신은 text-embedding-3-small 권장
   "input": "The food was delicious and the waiter..."
}
headers = {
   'Authorization': f'Bearer {OPENAI_API_KEY}',
   'Content-Type': 'application/json'
}

response = requests.post(url, headers=headers, json=payload)

print(response.status_code)
print(response.json())


200
{'object': 'list', 'data': [{'object': 'embedding', 'index': 0, 'embedding': [0.0022756963, -0.009305916, 0.015742613, -0.0077253063, -0.0047450014, 0.014917395, -0.009807394, -0.038264707, -0.0069127847, -0.028590616, 0.025251659, 0.018116701, -0.0036309576, -0.02554366, 0.00055543496, -0.016428178, 0.02828592, 0.0054083494, 0.009610611, -0.016415482, -0.015412526, 0.004272088, 0.0069953064, -0.007223828, -0.0039007403, 0.018573744, 0.008734611, -0.022699833, 0.011508612, 0.023893224, 0.015602961, -0.0035706533, -0.034963835, -0.0041514793, -0.026178442, -0.02150644, -0.0057066972, 0.011768873, 0.008455306, 0.004129262, 0.019157745, -0.014358787, 0.008982176, 0.0063605234, -0.04570436, 0.017900875, -0.005570219, -0.0007716578, -0.02215392, -0.0039229575, 0.02101131, -0.017608874, -0.011699047, -0.02256018, 0.01633931, 0.017164527, -0.00838548, 0.0015901309, 0.02507392, -0.024997747, 0.007807828, 0.005798741, -0.022115832, 0.002948566, -0.0061764363, -0.025556356, -0.008074437, 0.0

In [ ]:
def get_response(prompt):
    url = "https://api.openai.com/v1/embeddings"

    payload = json.dumps({
    "model": "text-embedding-ada-002",
    "input": prompt
    })
    headers = {
    'Authorization': f'Bearer {OPENAI_API_KEY}',
    'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    re_json = json.loads(response.text)

    return re_json["data"][0]["embedding"]

In [ ]:
print(len(user_data))

776369


In [ ]:

CHUNK = 10000
keys = list(user_data.keys())
chunks_list = []


for start in range(0, len(keys), CHUNK):
    sub_keys = keys[start:start + CHUNK]
    sub_dict = {k: user_data[k] for k in sub_keys}
    chunks_list.append(sub_dict)

for i, chunk in enumerate(chunks_list):
    globals()[f'chunks{i}'] = chunk

print(f"데이터를 총 {len(chunks_list)}개의 청크(chunks0 ~ chunks{len(chunks_list)-1})로 분할하고 동적 할당 완료.")

데이터를 총 78개의 청크(chunks0 ~ chunks77)로 분할하고 동적 할당 완료.


In [2]:
import os, time, pickle, math, random
from typing import Dict, Iterable, List
from tqdm import tqdm


from openai import OpenAI

client = OpenAI()

MODEL = "text-embedding-3-small"
BATCH_SIZE = 100
MAX_LEN = 15000
OUT_DIR = "handled"

os.makedirs(OUT_DIR, exist_ok=True)

def _trim(txt: str, max_len: int = MAX_LEN) -> str:
    if txt is None:
        return ""
    return txt[:max_len-1] if len(txt) > max_len else txt

def _retry_delay(attempt: int) -> float:
    base = min(60, 2 ** attempt)
    return base * (0.5 + random.random() * 0.5)

def embed_in_batches(user_data: Dict[str, str], out_path: str):
    """
    user_data: {user_id: text}
    out_path : 'handled/user_embX.pkl'
    """

    user_emb = {}
    if os.path.exists(out_path):
        try:
            with open(out_path, "rb") as f:
                user_emb = pickle.load(f)
            print(f"[resume] loaded {len(user_emb)} embeddings from {out_path}")
        except Exception as e:
            print("[resume] failed to load existing file:", e)


    keys = []
    texts = []
    for k, v in user_data.items():
        if k in user_emb:
            continue
        keys.append(k)
        texts.append(_trim(v))

    if not keys:
        print("[done] nothing to do; all embeddings already exist.")
        return

    total = len(keys)
    print(f"[start] to-embed: {total}, batch_size={BATCH_SIZE}")


    pbar = tqdm(total=total, desc="Embedding", unit="item")
    for start in range(0, total, BATCH_SIZE):
        end = min(start + BATCH_SIZE, total)
        batch_keys = keys[start:end]
        batch_texts = texts[start:end]

        attempt = 0
        while True:
            try:
                resp = client.embeddings.create(
                    model=MODEL,
                    input=batch_texts
                )

                for k, item in zip(batch_keys, resp.data):
                    user_emb[k] = item.embedding
                pbar.update(len(batch_keys))
                break
            except Exception as e:
                attempt += 1
                delay = _retry_delay(attempt)
                print(f"[warn] batch {start}-{end} failed (attempt {attempt}): {e}")
                print(f"      retry in {delay:.1f}s")
                time.sleep(delay)

                if attempt >= 6:
                    print("[error] giving up this batch; partial will be saved.")
                    break


            if (start // BATCH_SIZE) % 5 == 0 and len(user_emb) > 0:
                 try:
                     with open(out_path, "wb") as f:
                         pickle.dump(user_emb, f)

                 except Exception as e:
                     print("[checkpoint] save failed:", e)


    pbar.close()


    with open(out_path, "wb") as f:
        pickle.dump(user_emb, f)
        f.flush()
        os.fsync(f.fileno())
    print(f"[done] saved {len(user_emb)} embeddings -> {out_path}")
    time.sleep(3)



num_chunks = len([k for k in globals() if k.startswith('chunks') and isinstance(globals()[k], dict)])
print(f"\n*** 임베딩 작업 시작 (총 {num_chunks}개 청크) ***")

for i in range(0,100):
    data_var = f"chunks{i}"

    out_path = os.path.join(OUT_DIR, f"user_emb{i}.pkl")
    print(f"\n=== Part {i}: {data_var} -> {out_path} ===")

    embed_in_batches(globals()[data_var], out_path)

print("\n모든 임베딩 작업이 완료되었습니다.")

In [ ]:

import glob, re, pickle

pattern = "handled/user_emb[0-9]*.pkl"
files = glob.glob(pattern)


files = sorted(files, key=lambda p: int(re.search(r'(\d+)(?=\.pkl$)', p).group(1)))

if not files:
    raise FileNotFoundError(f"분할 파일을 찾지 못했습니다: {pattern}")

merged_user_emb = {}
seen = set()
sizes = []

for f in files:
    print(f"[LOAD] {f}")
    with open(f, "rb") as fr:
        part = pickle.load(fr)
    if not isinstance(part, dict):
        raise TypeError(f"{f} 는 dict가 아닙니다: {type(part)}")
    sizes.append(len(part))

    overlap = seen.intersection(part.keys())
    if overlap:
        ex = next(iter(overlap))
        print(f"  [WARN] 중복 키 {len(overlap)}개 발견 (예: {ex}) -> 이후 파일 값으로 덮어씀")

    merged_user_emb.update(part)
    seen.update(part.keys())

print(f"[MERGED] parts: {len(files)}, part sizes: {sizes}, total unique users: {len(merged_user_emb)}")

# ── 최종 저장 ──
out_path = "handled/user_emb.pkl"
with open(out_path, "wb") as fw:
    pickle.dump(merged_user_emb, fw, protocol=pickle.HIGHEST_PROTOCOL)
print(f"[SAVE] {out_path} complete!")


/content/drive/MyDrive/LLM_ESR_2023_v3/data/beauty
[LOAD] handled/user_emb0.pkl
[LOAD] handled/user_emb1.pkl
[LOAD] handled/user_emb2.pkl
[LOAD] handled/user_emb3.pkl
[LOAD] handled/user_emb4.pkl
[LOAD] handled/user_emb5.pkl
[LOAD] handled/user_emb6.pkl
[LOAD] handled/user_emb7.pkl
[LOAD] handled/user_emb8.pkl
[LOAD] handled/user_emb9.pkl
[LOAD] handled/user_emb10.pkl
[LOAD] handled/user_emb11.pkl
[LOAD] handled/user_emb12.pkl
[LOAD] handled/user_emb13.pkl
[LOAD] handled/user_emb14.pkl
[LOAD] handled/user_emb15.pkl
[LOAD] handled/user_emb16.pkl
[LOAD] handled/user_emb17.pkl
[LOAD] handled/user_emb18.pkl
[LOAD] handled/user_emb19.pkl
[LOAD] handled/user_emb20.pkl
[LOAD] handled/user_emb21.pkl
[LOAD] handled/user_emb22.pkl
[LOAD] handled/user_emb23.pkl
[LOAD] handled/user_emb24.pkl
[LOAD] handled/user_emb25.pkl
[LOAD] handled/user_emb26.pkl
[LOAD] handled/user_emb27.pkl
[LOAD] handled/user_emb28.pkl
[LOAD] handled/user_emb29.pkl
[LOAD] handled/user_emb30.pkl
[LOAD] handled/user_emb31.pkl

In [ ]:

import pickle
import numpy as np
with open("handled/user_emb.pkl", "rb") as f:
    user_emb = pickle.load(f)   # dict {user_id: emb_vector}

emb_list = []
users_sorted = sorted(user_emb.keys())
for u in users_sorted:
    emb_list.append(user_emb[u])

emb_list = np.array(emb_list)
with open("handled/user_emb_np.pkl", "wb") as f:
    pickle.dump(emb_list, f)
print(f"[SAVE] user_emb_np complete!")

/content/drive/MyDrive/LLM_ESR_2023_v3/data/movie
[SAVE] user_emb_np complete!
